# Use the Google Earth Engine API to retrieve Sentinel-2 surface reflectance, calculate spectral indices, and extract values at VT occurrences
#### 0. Set up authentification key (Requires GEE account, one-time execution)

In [ ]:
#!earthengine authenticate # Uncomment to execute, only needs to be done once.

## 1. Import and initialize Google Earth Engine and helpers

In [1]:
import ee
ee.Initialize()

In [2]:
# Retrieve GEE acount information
project_credentials = !earthengine ls
# Extract user name
user_name = project_credentials.s.split("/")[-1]
print(user_name)

lassekee


In [3]:
from src.data import eehelpers as eeh

## 2. Load Sentinel-2A imagery
##### Define date filter
<span style="color:red">Future idea: include additional indices and points in time? E.g., snow indices in spring as a potential predictor for snow bed vegetation?</span>

In [4]:
# Growing seasons (JJA) in available time window
"""
date_filter_gs2017 = ee.Filter.date('2017-06-01', '2017-08-31')
date_filter_gs2018 = ee.Filter.date('2018-06-01', '2018-08-31')
date_filter_gs2019 = ee.Filter.date('2019-06-01', '2019-08-31')
date_filter_gs2020 = ee.Filter.date('2020-06-01', '2020-08-31')
date_filter_gs2021 = ee.Filter.date('2021-06-01', '2021-08-31')

# Combine all dates to form multi year composite
date_filter_gs2017to2021 = ee.Filter.Or(date_filter_gs2017,
                                        date_filter_gs2018,
                                        date_filter_gs2019,
                                        date_filter_gs2020,
                                        date_filter_gs2021)
#.filter(date_filter_gs2017to2021) \
"""
# Define date filter
start_date_str = "2017-06-01"  # First day to use, inclusive
end_date_str = "2021-09-01"  # Last day to use, exclusive
# Months to subset, growing season
gs_start_month = 6  # Corresponds to June
gs_end_month = 8  # Corresponds to August

##### Retrieve predefined simple Norway geometry to mask tiles

In [5]:
nor_geometry = eeh.get_norway_geometry()

#### Retrieve Sentinel-2 surface reflectance data

In [6]:
# Filter dates, clouds, bounds, and calculate spectral indices.
collection_gs2017to2021 = ee.ImageCollection('COPERNICUS/S2_SR') \
.filterDate(start_date_str, end_date_str) \
.filter(ee.Filter.calendarRange(gs_start_month, gs_end_month, 'month')) \
.filterBounds(nor_geometry) \
.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
.map(eeh.mask_clouds_sentinel2) \
.map(eeh.clip_to_nor_geometry)

#### Calculate 5th and 95th quantiles to filter outliers

In [8]:
bottom_quantile_img, top_quantile_img = \
eeh.get_lower_and_upper_quantiles(
    img_collection=collection_gs2017to2021,
    lower=5, 
    upper=95
)

#### Replace outliers in each image with quantile values

In [9]:
collection_gs2017to2021 = collection_gs2017to2021 \
.map(lambda img: img.updateMask(img.lt(top_quantile_img))) \
.map(lambda img: img.unmask(top_quantile_img)) \
.map(lambda img: img.updateMask(img.gt(bottom_quantile_img))) \
.map(lambda img: img.unmask(bottom_quantile_img))

#### Calculate spectral indices for each filtered images

In [10]:
collection_gs2017to2021 = collection_gs2017to2021 \
.map(eeh.map_ndvi_sentinel2) \
.map(eeh.map_gndvi_sentinel2) \
.map(eeh.map_evi_sentinel2) \
.map(eeh.map_savi_sentinel2) \
.map(eeh.map_ndmi_sentinel2)

In [11]:
# Print number of images in collection
collection_gs2017to2021.size().getInfo()

6790

In [14]:
# Also load names of spectral indices
import json

with open('../data/dict/spectral_indices.json') as json_file:
    indices_dict = json.load(json_file)
    
band_names = collection_gs2017to2021.first().bandNames().getInfo()
print("Band names:")
_ = [print(x + "; ", end="") for x in band_names]

spectral_index_names = [key for key in indices_dict.keys()]
print("\nSpectral indices:")
_ = [print(x + "; ", end="") for x in spectral_index_names]

Band names:
B1; B2; B3; B4; B5; B6; B7; B8; B8A; B9; B11; B12; NDVI; GNDVI; EVI; SAVI; NDMI; 
Spectral indices:
NDVI; GNDVI; EVI; SAVI; NDMI; 

---
#### Create median composite of spectral indices

In [15]:
# Create a median composite image, but keep (and rename) only the spectral indices
gs2017to2021_out_img = \
collection_gs2017to2021.median().select(
    band_names,
    [idx_name + "_median_comp" for idx_name in band_names]  # Rename to indicate it stems from median composite
)

In [16]:
gs2017to2021_out_img.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1_median_comp',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 6.553500175476074},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B2_median_comp',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 6.553500175476074},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B3_median_comp',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 6.553500175476074},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B4_median_comp',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 6.553500175476074},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B5_median_comp',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 6.553500175476074},
   'crs': 'EPSG:4326',
   'crs_transf

---
#### Create greenest pixel composite based on NDVI

In [17]:
gs2017to2021_out_img = gs2017to2021_out_img.addBands(
    collection_gs2017to2021.qualityMosaic('NDVI').select(
        spectral_index_names,
        [idx_name + "_greenest_pixel" for idx_name in spectral_index_names]
    )
)

In [18]:
gs2017to2021_out_img.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1_median_comp',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 6.553500175476074},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B2_median_comp',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 6.553500175476074},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B3_median_comp',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 6.553500175476074},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B4_median_comp',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 6.553500175476074},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B5_median_comp',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 6.553500175476074},
   'crs': 'EPSG:4326',
   'crs_transf

---
---
---

### Visualize results (optional)

##### Pick map to visualize

In [19]:
img_to_map = gs2017to2021_out_img

##### Prepare folium library

In [20]:
# Library to display WMS map in notebook
import folium

In [21]:
# Add EE drawing method to folium, i.e. overwrite method as a monkey patch
folium.Map.add_ee_layer = eeh.add_ee_layer

In [22]:
# Calculate center point of polygon
geom_centroid = nor_geometry.centroid(100).getInfo().get('coordinates')
print(geom_centroid)

[13.209238450857365, 64.68724893483645]


### RGB composite on streetmap

In [23]:
# Set visualization parameters, RGB
vis_rgb = {'bands': ['B4', 'B3', 'B2'], 'max': 0.3}

In [24]:
eeMap_rgb = folium.Map(location=geom_centroid[::-1], zoom_start=4, height='100%', width='100%')

In [25]:
eeMap_rgb.add_ee_layer(collection_gs2017to2021.median(), vis_rgb, 'Norway Sentinel-2 RBG')

In [26]:
display(eeMap_rgb) # Display map, may take a while to load!

##### Visualize spectral index

In [27]:
vis_idx_name = "NDVI_greenest_pixel"

In [30]:
# Visualization parameters for map representation 
# (display values from -1 to 1, blue areas = -1, white = 0, green = 1)
ndvi_vis = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']};
# Select spectral index band defined above
vis_idx_img = img_to_map.select(vis_idx_name)

In [31]:
# Create map instance
eeMap_ndvi = folium.Map(location=geom_centroid[::-1], zoom_start=4, height='100%', width='100%')

In [32]:
# Display NDVI image with chosen display parameters
eeMap_ndvi.add_ee_layer(vis_idx_img, ndvi_vis, f'Norway Sentinel2 - {vis_idx_name}')

In [33]:
# Display map
display(eeMap_ndvi) # Obs! It may take a while to load the tiles!

---------
## Extract Values at Vegetation Type locations

#### Retrieve Vegetation Type data

In [36]:
# Define the image you want to use for extraction
img_to_extract = gs2017to2021_out_img
gs2017to2021_out_img.bandNames().getInfo()

['B1_median_comp',
 'B2_median_comp',
 'B3_median_comp',
 'B4_median_comp',
 'B5_median_comp',
 'B6_median_comp',
 'B7_median_comp',
 'B8_median_comp',
 'B8A_median_comp',
 'B9_median_comp',
 'B11_median_comp',
 'B12_median_comp',
 'NDVI_median_comp',
 'GNDVI_median_comp',
 'EVI_median_comp',
 'SAVI_median_comp',
 'NDMI_median_comp',
 'NDVI_greenest_pixel',
 'GNDVI_greenest_pixel',
 'EVI_greenest_pixel',
 'SAVI_greenest_pixel',
 'NDMI_greenest_pixel']

In [38]:
# Initialize VT FeatureCollection
vt_presence_fc = eeh.VTFeatureCollection(
    user_name=user_name,
    vt_asset_name='vtprespoints',
    extract_columns=['POINT_X','POINT_Y','FLATE_NR','layer']
)

FeatureCollection initialized, number of rows: 25679


#### Extract image data
Use 30 m spatial scale to make data set size manageable and match Landsat-7 scale.</br>
_Increase tileScale (default:1) in 2^n steps (2, 4, 8,...) if computation times out!_

In [42]:
sampled_sent_list = vt_presence_fc.extract_from_eeimg(
    img=img_to_extract,
    scale=30,
    tile_scale=8
) 

Starting extraction. This process can take 5-10 minutes.


EEException: User memory limit exceeded.

In [ ]:
# Print first entry for testing
print(sampled_sent_list[0])

<h3 style="color:salmon">Create result DataFrame and save as csv</h3>

In [ ]:
feature_mat = \
eeh.df_from_sampled_list(
    sampled_list=sampled_sent_list,
    saveas="sent2_gs17to21_median_and_maxndvi.csv",
    drop_cols=["type", "geometry", "id"]
)

In [ ]:
# Print results
print(feature_mat.shape)
feature_mat.head()

<h2 style="color:salmon;font-weight:bold">The end.</h2>

---
---
---
---
---

---
## Data visualization

In [ ]:
data_df = feature_mat.drop(["FLATE_NR", "POINT_X", "POINT_Y", "layer"], axis=1)

# Round data
for col_name in data_df.columns:
    data_df[col_name] = data_df[col_name].apply(lambda x: round(x, 4))
    
data_df.head()

In [ ]:
# Print mean and ranges for each column to get some feeling for the data
import statistics

for col in data_df.columns:
    print(f"{col} -- mean={statistics.mean(data_df[col])} ; min={min(data_df[col])} ; max={max(data_df[col])}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sr_data_df = data_df#.drop(["B6_median_comp"], axis=1)

save_path = "../results/plots/features/satellite_data/sentinel2/"

fig, ax = plt.subplots(figsize=(12, 8), dpi=300)
ax.set_title("Sentinel 2 - pixel values at Vegetation Type\npresence point locations", fontsize=20)
ax.set_xticks(ax.get_xticks())
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
ax.tick_params(labelsize=18)
ax.set_ylabel("Surface reflectance / spectral index", fontsize=18)
ax.set_ylim(-1, 1.5)

sns.violinplot(data=sr_data_df, ax=ax)

fig.savefig(f"{save_path}sentinel_pixels_violin.jpg",
            bbox_inches = 'tight',
            pad_inches=0)

In [ ]:
# Plot histograms
import matplotlib.pyplot as plt
import numpy as np

save_path = "../results/plots/features/satellite_data/sentinel2/"

for col in data_df.columns:
    
    hist, bins = np.histogram(data_df[col], bins=50)
    width = 0.7 * (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:]) / 2

    fig, ax = plt.subplots(figsize=(12,12), dpi=300)
    ax.bar(center, hist, align='center', width=width)

    if (col in [x+"_median_comp" for x in spectral_index_names]) or (col in [x+"_greenest_pixel" for x in spectral_index_names]):
        ax.set_xlabel(col, fontsize=32)
    else:
        ax.set_xlabel("Surface reflectance", fontsize=32)

    ax.set_ylabel("Frequency", fontsize=32)
    ax.tick_params(labelsize=22)

    fig.savefig(f"{save_path}{col}_histogram.jpg",
                bbox_inches = 'tight',
                pad_inches=0)